In [78]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
#from catboost import CatBoostRegressor
import re

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [79]:
url_train="https://drive.google.com/u/0/uc?id=1RhXDaw02wlVtvUggA9C81i78vDTM8u14&export=download"
train = pd.read_csv(url_train) # мой подготовленный датасет для обучения модели
#test = pd.read_csv('test.csv')
#sample_submission = pd.read_csv('sample_submission.csv')

In [80]:
#Смотрим собранные Юрием данные
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37426 entries, 0 to 37425
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37426 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37426 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37128 non-null  object 
 9   enginePower              37410 non-null  object 
 10  description              37426 non-null  object 
 11  mileage                  37426 non-null  int64  
 12  Привод                   37356 non-null  object 
 13  Руль                     37232 non-null  object 
 14  Состояние             

In [81]:
#Имеем 31 колонку и 37426 строчек
#Посмотрим как данные выглядят и может сразу что то уберем
pd.set_option('display.max_columns', 40)
display(train.head(20))

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Аудиосистема,Фары,Климат-контроль,Салон,Диски,autoCatalogUrl,itemPrice,marketPrice,status,closingReason,typeOfTrade,canonicalUrl,avitoId
0,внедорожник,Land Rover,Чёрный,Бензин,Discovery,5.0,1995,механическая,3.9,182 л.с.,Прoдaю лeгендаpный aвтомобиль по ценe китайcко...,180000,полный,Левый,Не битый,3,передние и задние,гидро-,NaN,NaN,Кондиционер,велюр,NaN,/autocatalog/land_rover/discovery/i-19891998_4...,170000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/land_ro...,263245368
1,внедорожник,Hyundai,Зелёный,Бензин,Galloper,5.0,1997,автоматическая,3.0,141 л.с.,"В нормальном состоянии. Салон кожаный, пяти м...",330000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/hyundai/galloper/i-19912003/vnedo...,170000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/hyundai...,633133188
2,седан,Mercedes-Benz,Чёрный,Бензин,S-класс,4.0,2008,автоматическая,5.5,388 л.с.,"Отличный автомобиль, всё есть, надо небольшое ...",124000,полный,Левый,Не битый,2,передние и задние,электрогидро-,8+ колонок,NaN,NaN,NaN,"18""",/autocatalog/mercedes-benz/s-klass/w221-200520...,700000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,624926344
3,хэтчбек,Skoda,Белый,Бензин,Octavia,5.0,2013,робот,1.4,140 л.с.,"В отличном состоянии, комплетация Elegance c д...",76500,передний,Левый,Не битый,1,передние и задние,электро-,NaN,NaN,климат-контроль многозонный,NaN,"16""",/autocatalog/skoda/octavia/iii-20132017_3291/k...,675000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/skoda_o...,624923120
4,микроавтобус,Mercedes-Benz,Чёрный,Бензин,Sprinter,4.0,2019,автоматическая,3.5,258 л.с.,"B наличии, бeз пpобeга Меrсеdеs Sprintеr VIP о...",90,задний,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,кожа,NaN,/autocatalog/mercedes-benz/sprinter_2370/w906-...,10900000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,622789649
5,минивэн,Opel,Серый,Бензин,Zafira,5.0,2012,робот,1.8,140 л.с.,"В хорошем состоянии, находится в сочи",111300,передний,Левый,Не битый,2,передние и задние,электро-,4 колонки,NaN,климат-контроль однозонный,NaN,"16""",/autocatalog/opel/zafira/b-restayling-20082014...,385000,444200.0,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/opel_za...,633420571
6,седан,Mercedes-Benz,Чёрный,Дизель,E-класс,4.0,2004,автоматическая,2.1,150,....ПРИГНАНА ИЗ ГЕРМАНИИ 2008Г...ЖЕЛЕЗО И КРАС...,380000,задний,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/mercedes-benz/e-klass/w211s211-20...,340000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,634877947
7,седан,Oldsmobile,Красный,Бензин,Eighty-Eight,4.0,1992,автоматическая,3.8,172 л.с.,Машина не нa ходу. Kоробка в идeалe! двиготeль...,150000,передний,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/oldsmobile/eighty-eight/x-1992199...,80000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/oldsmob...,621276255
8,внедорожник,Mercedes-Benz,Золотой,Бензин,M-класс AMG,5.0,2002,автоматическая,5.4,347,Пpодaм Mеpсeдес Мл 320 АMG в кузовe (w163) тех...,160000,полный,Левый,Не битый,1,передние и задние,гидро-,NaN,NaN,NaN,NaN,NaN,/autocatalog/mercedes-benz/m-klass_amg/w163-re...,500000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,627205114
9,внедорожник,Volkswagen,Серый,Дизель,Touareg,5.0,2008,автоматическая,3.0,240 л.с.,Сpoчнo!!! Maшина в отличном техничеcком cоcтоя...,222000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/volkswagen/touareg/i-restayling-2...,715000,796600.0,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/volkswa...,629324840


In [82]:
#На первый взгляд видим следующее:
#1. enginePower- нужно будет убрать "л.с."
#2. description- предлагаю полностью убрать это столбец
#3. autoCatalogUrl, status, closingReason, canonicalUrl, avitoId, Аудиосистема, Фары - можно спокойно удалять. Такие значения
# как аудиосистема или фары, практически не вляют на ценник, да и информации по ним мало.
#4. С остальными полями будем разбираться дальше.
train.drop(['autoCatalogUrl', 'status', 'closingReason','canonicalUrl','avitoId','Аудиосистема','Фары', 'description'], inplace = True, axis = 1)

In [83]:
#Проверям, смотрим. Осталось 23 колонки. Теперь пройдемся по каждой
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37426 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37426 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37426 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37128 non-null  object 
 9   enginePower              37410 non-null  object 
 10  mileage                  37426 non-null  int64  
 11  Привод                   37356 non-null  object 
 12  Руль                     37232 non-null  object 
 13  Состояние                37426 non-null  object 
 14  Владельцы             

In [84]:
#Посмотрим какие уникальные значения в столбце bodyType
train.bodyType.value_counts()

седан           14412
внедорожник      9808
хэтчбек          6198
минивэн          1926
универсал        1693
фургон           1591
купе              834
пикап             354
микроавтобус      347
кабриолет         193
Name: bodyType, dtype: int64

In [85]:
#все хорошо есть, но есть 70 пропусков, предлагаю их убрать. Еще одно НО, посмотрим на них полностью
temp = train[train.bodyType.isnull()]
display(temp.head(30))

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
1068,NaN,NaN,Фиолетовый,NaN,NaN,NaN,2012,NaN,NaN,117 л.с.,43750,NaN,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,650000,NaN,Продаю личный автомобиль
1234,NaN,NaN,Серебряный,NaN,NaN,NaN,2007,NaN,NaN,248 л.с.,195000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,399000,497400.0,Продаю личный автомобиль
1380,NaN,NaN,Красный,NaN,NaN,NaN,1994,NaN,NaN,90 л.с.,198500,NaN,Левый,Не битый,4+,NaN,NaN,NaN,NaN,NaN,87000,94700.0,Автомобиль приобретён на продажу
1448,NaN,NaN,Белый,NaN,NaN,NaN,2011,NaN,NaN,163 л.с.,221000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,810000,957700.0,Продаю личный автомобиль
1480,NaN,NaN,Чёрный,NaN,NaN,NaN,2012,NaN,NaN,170 л.с.,143000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,1000000,974300.0,Продаю личный автомобиль
1484,NaN,NaN,Чёрный,NaN,NaN,NaN,2013,NaN,NaN,249 л.с.,77000,NaN,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,1100000,1109800.0,Продаю личный автомобиль
1818,NaN,NaN,Чёрный,NaN,NaN,NaN,2014,NaN,NaN,245 л.с.,77000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,1630000,NaN,Продаю личный автомобиль
2141,NaN,NaN,Серый,NaN,NaN,NaN,2001,NaN,NaN,65 л.с.,202000,NaN,Правый,Не битый,2,NaN,NaN,NaN,NaN,NaN,80000,NaN,Продаю личный автомобиль
2193,NaN,NaN,Чёрный,NaN,NaN,NaN,2016,NaN,NaN,135 л.с.,35000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,599000,572000.0,Продаю личный автомобиль
2197,NaN,NaN,Синий,NaN,NaN,NaN,2006,NaN,NaN,125 л.с.,156000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,250000,249200.0,Автомобиль приобретён на продажу


In [86]:
#Видим, что информации по ним 0, удаляем
train.dropna(subset = ['bodyType'],inplace = True)

In [87]:
#число полей сократилось на 70
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37356 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37356 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37356 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37122 non-null  object 
 9   enginePower              37340 non-null  object 
 10  mileage                  37356 non-null  int64  
 11  Привод                   37356 non-null  object 
 12  Руль                     37162 non-null  object 
 13  Состояние                37356 non-null  object 
 14  Владельцы             

In [88]:
#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10}

In [89]:
#Заменяем
train['bodyType'] = train['bodyType'].map(dict_bodyType)

In [90]:
#Смотрим
train.head()

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
0,2,Land Rover,Чёрный,Бензин,Discovery,5.0,1995,механическая,3.9,182 л.с.,180000,полный,Левый,Не битый,3,передние и задние,гидро-,Кондиционер,велюр,NaN,170000,NaN,Продаю личный автомобиль
1,2,Hyundai,Зелёный,Бензин,Galloper,5.0,1997,автоматическая,3.0,141 л.с.,330000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,170000,NaN,Продаю личный автомобиль
2,1,Mercedes-Benz,Чёрный,Бензин,S-класс,4.0,2008,автоматическая,5.5,388 л.с.,124000,полный,Левый,Не битый,2,передние и задние,электрогидро-,NaN,NaN,"18""",700000,NaN,Продаю личный автомобиль
3,3,Skoda,Белый,Бензин,Octavia,5.0,2013,робот,1.4,140 л.с.,76500,передний,Левый,Не битый,1,передние и задние,электро-,климат-контроль многозонный,NaN,"16""",675000,NaN,Продаю личный автомобиль
4,9,Mercedes-Benz,Чёрный,Бензин,Sprinter,4.0,2019,автоматическая,3.5,258 л.с.,90,задний,Левый,Не битый,1,NaN,NaN,NaN,кожа,NaN,10900000,NaN,Продаю личный автомобиль


# Смотрим "brand"

In [91]:
#Получаем, что у нас 123 различных брендов. Пойдем по пути, что в ТЕСТОВОМ у нас одни BMW
train.brand.value_counts()

Mercedes-Benz    3002
BMW              2519
Volkswagen       2375
Hyundai          2335
ВАЗ (LADA)       2302
                 ... 
McLaren             1
Doninvest           1
Dacia               1
AMC                 1
Lotus               1
Name: brand, Length: 123, dtype: int64

In [92]:
#Оставим одни значения BMW
#train_BMW = train[train['brand'].isin(['BMW'])]

# Обработка категориальных признаков "скопом"

# color
# vehicleTransmission
# Привод
# Состояние
# Руль
# Name
# fuelType
# typeOfTrade

In [93]:
#В столбце "Руль" есть пропуски, убираем.
print(train.Руль.value_counts())
print(train.Руль.isna().sum())
fs = train[train.Руль.isnull()]

Левый     36017
Правый     1145
Name: Руль, dtype: int64
194


In [94]:
#Посмотрим в каких брендах у нас пропуски и мы точно знаем, что они имеют "левый" руль
fs.brand.value_counts()

Hyundai          16
Ford             14
Volkswagen       13
KIA              13
Mercedes-Benz    12
ВАЗ (LADA)       12
Renault          10
Nissan           10
Peugeot           9
Land Rover        8
Skoda             8
Chevrolet         8
Toyota            7
BMW               7
Volvo             6
Mazda             6
Mitsubishi        6
Opel              5
Audi              5
Lexus             4
Citroen           2
Chery             2
Datsun            2
SsangYong         2
УАЗ               2
Infiniti          2
Jeep              1
Daewoo            1
Subaru            1
Name: brand, dtype: int64

In [95]:
train.loc[(train['brand'] == "BMW") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Renault") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Volkswagen") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Peugeot") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Chevrolet") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Audi") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Opel") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Lexus") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Skoda") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Mercedes-Benz") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train.Руль.isnull()) , 'Руль'] = "Правый"

In [96]:
#Нашел статью на хабре, пробуем https://habr.com/ru/post/456294/  , а то в прошлый раз все словарями делал. 
LE = LabelEncoder()
LE.fit(train['color'])
train['Color_new'] = LE.transform(train['color'])
train.drop(['color'],axis = 1, inplace = True)
LE.fit(train['vehicleTransmission'])
train['vehicleTransmission_new'] = LE.transform(train['vehicleTransmission'])
train.drop(['vehicleTransmission'],axis = 1, inplace = True)
LE.fit(train['Привод'])
train['Привод_new'] = LE.transform(train['Привод'])
train.drop(['Привод'],axis = 1, inplace = True)
LE = LabelEncoder()
#LE.fit(train['Руль']) Выдает ошибку, может кто то поймет как с ней бороться
#train['Руль_new'] = LE.transform(train['Руль'])
#train.drop(['Руль'],axis = 1, inplace = True)
LE.fit(train['Состояние'])
train['Состояние_new'] = LE.transform(train['Состояние'])
train.drop(['Состояние'],axis = 1, inplace = True)
LE.fit(train['name'])
train['name_new'] = LE.transform(train['name'])
train.drop(['name'],axis = 1, inplace = True)
LE.fit(train['fuelType'])
train['fuelType_new'] = LE.transform(train['fuelType'])
train.drop(['fuelType'],axis = 1, inplace = True)
LE.fit(train['typeOfTrade'])
train['typeOfTrade_new'] = LE.transform(train['typeOfTrade'])
train.drop(['typeOfTrade'],axis = 1, inplace = True)

In [97]:
dict_rul={"Левый": 1, "Правый": 2}
train['Руль'] = train['Руль'].map(dict_rul)

# Столбец "enginePower"

In [98]:
#Чистим от л.с.
train['enginePower'] = train['enginePower'].apply(lambda x: float(str(x).split()[0]))

In [99]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37356 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  int64  
 1   brand                    37356 non-null  object 
 2   numberOfDoors            37355 non-null  float64
 3   productionDate           37356 non-null  int64  
 4   engineDisplacement       37122 non-null  object 
 5   enginePower              37340 non-null  float64
 6   mileage                  37356 non-null  int64  
 7   Руль                     37356 non-null  int64  
 8   Владельцы                37356 non-null  object 
 9   Электростеклоподъемники  5839 non-null   object 
 10  Усилитель руля           5808 non-null   object 
 11  Климат-контроль          3310 non-null   object 
 12  Салон                    2219 non-null   object 
 13  Диски                    4922 non-null   object 
 14  itemPrice             

# Столбец "engineDisplacement"

In [100]:
#Видим пропуски, но не вижу пока способа их подставить.
print(train.engineDisplacement.isna().sum())

234


In [101]:
train.dropna(subset = ['engineDisplacement'],inplace = True)

In [102]:
train["engineDisplacement"].unique()

array(['3.9', '3.0', '5.5', '1.4', '3.5', '1.8', '2.1', '3.8', '5.4',
       '2.4', '3.2', '4.5', '2.0', '1.6', '2.9', '2.5', '4.8', '2.2',
       '4.4', '2.7', '2.3', '3.3', '4.0', '6.0+', '5.9', '1.5', '3.6',
       '2.8', '1.7', '5.7', '4.2', '1.3', '0.8', '6.0', '5.8', '0.7',
       '4.1', '1.0', '5.6', '1.2', '4.6', '5.0', '3.7', '4.7', '2.6',
       '5.3', '3.1', '1.9', '1.1', '4.3', '5.2', '0.6', '3.4', '4.9',
       '0.9', '5.1'], dtype=object)

In [103]:
train.loc[(train["engineDisplacement"] == "6.0+") , 'engineDisplacement'] = "7.0"

In [104]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37122 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37122 non-null  int64  
 1   brand                    37122 non-null  object 
 2   numberOfDoors            37121 non-null  float64
 3   productionDate           37122 non-null  int64  
 4   engineDisplacement       37122 non-null  object 
 5   enginePower              37122 non-null  float64
 6   mileage                  37122 non-null  int64  
 7   Руль                     37122 non-null  int64  
 8   Владельцы                37122 non-null  object 
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

In [105]:
#Пока не знаю как округлить

In [106]:
#После того, как дропнули тут пустые значения, меняем тип данных у "enginePower"
train.enginePower = train.enginePower.astype(np.int32)

# Столбец "numberOfDoors" переводим в int

In [107]:
train.numberOfDoors.value_counts()

5.0    18323
4.0    15731
2.0     1898
3.0     1169
Name: numberOfDoors, dtype: int64

In [108]:
train[train.numberOfDoors.isna()]
train.dropna(subset = ['numberOfDoors'],inplace = True)

In [109]:
train.numberOfDoors = train.numberOfDoors.astype(np.int32)

In [110]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   brand                    37121 non-null  object 
 2   numberOfDoors            37121 non-null  int32  
 3   productionDate           37121 non-null  int64  
 4   engineDisplacement       37121 non-null  object 
 5   enginePower              37121 non-null  int32  
 6   mileage                  37121 non-null  int64  
 7   Руль                     37121 non-null  int64  
 8   Владельцы                37121 non-null  object 
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

# Столбец "Владельцы"

In [111]:
train.Владельцы.value_counts()
#Видим что есть 4+, изменим на просто 4

2     11146
1     10003
3      9841
4+     6131
Name: Владельцы, dtype: int64

In [112]:
train.loc[(train["Владельцы"] == "4+") , 'Владельцы'] = 4

In [113]:
train.Владельцы = train.Владельцы.astype(np.int32)

In [114]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   brand                    37121 non-null  object 
 2   numberOfDoors            37121 non-null  int32  
 3   productionDate           37121 non-null  int64  
 4   engineDisplacement       37121 non-null  object 
 5   enginePower              37121 non-null  int32  
 6   mileage                  37121 non-null  int64  
 7   Руль                     37121 non-null  int64  
 8   Владельцы                37121 non-null  int32  
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

# Столбцы 'Электростеклоподъемники', "Усилитель руля","Климат-контроль", "Салон", "Диски", "marketPrice"
Какие у вас предложения? оставляем или дропаем?

### Дропаю оставшиеся признаки

In [115]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   brand                    37121 non-null  object 
 2   numberOfDoors            37121 non-null  int32  
 3   productionDate           37121 non-null  int64  
 4   engineDisplacement       37121 non-null  object 
 5   enginePower              37121 non-null  int32  
 6   mileage                  37121 non-null  int64  
 7   Руль                     37121 non-null  int64  
 8   Владельцы                37121 non-null  int32  
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

In [116]:
train.drop(['Электростеклоподъемники', 'Усилитель руля', 'Климат-контроль','Салон','Диски','marketPrice'], inplace = True, axis = 1)

In [117]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   bodyType                 37121 non-null  int64 
 1   brand                    37121 non-null  object
 2   numberOfDoors            37121 non-null  int32 
 3   productionDate           37121 non-null  int64 
 4   engineDisplacement       37121 non-null  object
 5   enginePower              37121 non-null  int32 
 6   mileage                  37121 non-null  int64 
 7   Руль                     37121 non-null  int64 
 8   Владельцы                37121 non-null  int32 
 9   itemPrice                37121 non-null  int64 
 10  Color_new                37121 non-null  int64 
 11  vehicleTransmission_new  37121 non-null  int64 
 12  Привод_new               37121 non-null  int64 
 13  Состояние_new            37121 non-null  int64 
 14  name_new                 37121 non-nul

In [118]:
train.head()

,bodyType,brand,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,itemPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new
0,2,Land Rover,5,1995,3.9,182,180000,1,3,170000,15,2,2,1,445,0,1
1,2,Hyundai,5,1997,3.0,141,330000,1,3,170000,4,0,2,1,586,0,1
2,1,Mercedes-Benz,4,2008,5.5,388,124000,1,2,700000,15,0,2,1,983,0,1
3,3,Skoda,5,2013,1.4,140,76500,1,1,675000,1,3,1,1,836,0,1
4,9,Mercedes-Benz,4,2019,3.5,258,90,1,1,10900000,15,0,0,1,1069,0,1


### подумем что делать с графой brand

In [119]:
display(pd.DataFrame(train.brand.value_counts()))
print("Значений, встретившихся в столбце более 10 раз:", (train.brand.value_counts()>10).sum())
train.loc[:, ['brand']].info()

,brand
Mercedes-Benz,2979
BMW,2509
Volkswagen,2373
Hyundai,2330
ВАЗ (LADA),2300
...,...
Asia,1
Doninvest,1
Dacia,1
Renault Samsung,1


Значений, встретившихся в столбце более 10 раз: 66
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   brand   37121 non-null  object
dtypes: object(1)
memory usage: 580.0+ KB


преобразуем признак brand в dummy-переменные. Для этого сначала требуется сделать небольшую предобработку, чтобы сократить количество уникальных значений в этом признаке. Для начала, найдем бренды которые описывают больше половины (67%) примеров:

In [120]:
brands_with_freqs = list(train.brand.value_counts())

In [121]:
top_brands_count = int(np.percentile(brands_with_freqs, 67))
top_brands_count

70

Теперь оставим только этот топ, остальные города назовем `other`:

In [122]:
all_brands = train.brand.value_counts().index
top_brands = list(all_brands)[:top_brands_count]
brands_to_throw_away = list(set(all_brands) - set(top_brands))
train.loc[train['brand'].isin(brands_to_throw_away), 
             'brand'] = 'other'

Теперь количество уникальных значений в признаке `brand` сократилось до:

In [123]:
len(train.brand.value_counts())

71

Можем создать dummy-признаки для этой переменной:

In [124]:
#pd.get_dummies(train.brand, drop_first=True).head()

In [125]:
train.head()

,bodyType,brand,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,itemPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new
0,2,Land Rover,5,1995,3.9,182,180000,1,3,170000,15,2,2,1,445,0,1
1,2,Hyundai,5,1997,3.0,141,330000,1,3,170000,4,0,2,1,586,0,1
2,1,Mercedes-Benz,4,2008,5.5,388,124000,1,2,700000,15,0,2,1,983,0,1
3,3,Skoda,5,2013,1.4,140,76500,1,1,675000,1,3,1,1,836,0,1
4,9,Mercedes-Benz,4,2019,3.5,258,90,1,1,10900000,15,0,0,1,1069,0,1


In [126]:
brands_df = pd.get_dummies(train.brand, drop_first=True)  
train = pd.concat([train, brands_df],axis = 1)
train.head

<bound method NDFrame.head of        bodyType          brand  numberOfDoors  productionDate  \
0             2     Land Rover              5            1995   
1             2        Hyundai              5            1997   
2             1  Mercedes-Benz              4            2008   
3             3          Skoda              5            2013   
4             9  Mercedes-Benz              4            2019   
...         ...            ...            ...             ...   
37421         2     Volkswagen              5            2010   
37422         1            KIA              4            2015   
37423         1     Volkswagen              4            2014   
37424         1           Audi              4            2011   
37425         1       Infiniti              4            2010   

      engineDisplacement  enginePower  mileage  Руль  Владельцы  itemPrice  \
0                    3.9          182   180000     1          3     170000   
1                    3.0         

In [127]:
train.head()

,bodyType,brand,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,itemPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new,Alfa Romeo,Audi,BAW,...,SEAT,Saab,Skoda,Smart,SsangYong,Subaru,Suzuki,Toyota,Volkswagen,Volvo,Vortex,other,ВАЗ (LADA),ВИС,ГАЗ,ЗАЗ,ИЖ,Москвич,ТагАЗ,УАЗ
0,2,Land Rover,5,1995,3.9,182,180000,1,3,170000,15,2,2,1,445,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Hyundai,5,1997,3.0,141,330000,1,3,170000,4,0,2,1,586,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,Mercedes-Benz,4,2008,5.5,388,124000,1,2,700000,15,0,2,1,983,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Skoda,5,2013,1.4,140,76500,1,1,675000,1,3,1,1,836,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,Mercedes-Benz,4,2019,3.5,258,90,1,1,10900000,15,0,0,1,1069,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [128]:
#дропаем первоначальную колонку brand
train.drop(['brand'], inplace = True, axis = 1)

In [129]:
train.head()

,bodyType,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,itemPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new,Alfa Romeo,Audi,BAW,BMW,...,SEAT,Saab,Skoda,Smart,SsangYong,Subaru,Suzuki,Toyota,Volkswagen,Volvo,Vortex,other,ВАЗ (LADA),ВИС,ГАЗ,ЗАЗ,ИЖ,Москвич,ТагАЗ,УАЗ
0,2,5,1995,3.9,182,180000,1,3,170000,15,2,2,1,445,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,5,1997,3.0,141,330000,1,3,170000,4,0,2,1,586,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,4,2008,5.5,388,124000,1,2,700000,15,0,2,1,983,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,5,2013,1.4,140,76500,1,1,675000,1,3,1,1,836,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,4,2019,3.5,258,90,1,1,10900000,15,0,0,1,1069,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [130]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 86 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   bodyType                 37121 non-null  int64 
 1   numberOfDoors            37121 non-null  int32 
 2   productionDate           37121 non-null  int64 
 3   engineDisplacement       37121 non-null  object
 4   enginePower              37121 non-null  int32 
 5   mileage                  37121 non-null  int64 
 6   Руль                     37121 non-null  int64 
 7   Владельцы                37121 non-null  int32 
 8   itemPrice                37121 non-null  int64 
 9   Color_new                37121 non-null  int64 
 10  vehicleTransmission_new  37121 non-null  int64 
 11  Привод_new               37121 non-null  int64 
 12  Состояние_new            37121 non-null  int64 
 13  name_new                 37121 non-null  int64 
 14  fuelType_new             37121 non-nul

### переведем engineDisplacement из obj в float

In [131]:
train.engineDisplacement = train.engineDisplacement.astype(np.float)

In [132]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 86 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   numberOfDoors            37121 non-null  int32  
 2   productionDate           37121 non-null  int64  
 3   engineDisplacement       37121 non-null  float64
 4   enginePower              37121 non-null  int32  
 5   mileage                  37121 non-null  int64  
 6   Руль                     37121 non-null  int64  
 7   Владельцы                37121 non-null  int32  
 8   itemPrice                37121 non-null  int64  
 9   Color_new                37121 non-null  int64  
 10  vehicleTransmission_new  37121 non-null  int64  
 11  Привод_new               37121 non-null  int64  
 12  Состояние_new            37121 non-null  int64  
 13  name_new                 37121 non-null  int64  
 14  fuelType_new          

### попробуем построить простенькую модель

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression

In [134]:
X = train.drop(['itemPrice'], axis = 1)
y = train['itemPrice']

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [136]:
# нормализуем X_train
transformer = RobustScaler()
X_train = transformer.fit_transform(X_train)

In [137]:
# normalising X_test
transformer = RobustScaler()
X_test = transformer.fit_transform(X_test)

In [138]:
# training the model
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

In [139]:
# evaluating the model
from sklearn.metrics import mean_squared_error
mean_squared_error(np.exp(y_test), np.exp(y_pred))

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [140]:
y_pred

array([ 539904.69259473, 1326622.33499827, 1952510.73813585, ...,
        621892.34530985,  570308.59684137,  539089.19432669])

In [141]:
y_train

1251     1245000
183       840000
15544     205000
29522     550000
15822     120000
          ...   
33207     436000
25694     490000
7906      750000
18819    2130000
8883      300000
Name: itemPrice, Length: 29696, dtype: int64

In [142]:
from sklearn import metrics # инструменты для оценки точности модели

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))


MAE: 354263.7355203748


### Случайный лес

In [143]:
RANDOM_SEED = 42
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict

forest = RandomForestRegressor(n_estimators=10, random_state=RANDOM_SEED) #max_depth = 15, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100, verbose = 100)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')
print_regression_metrics(y_test, y_pred)

RMSE = 508087.97, MAE = 155086.02, R-sq = 0.70, MAPE = 0.35 


### Линейная регрессия еще раз

In [144]:
# training the model
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

print_regression_metrics(y_test, y_pred)

RMSE = 647477.11, MAE = 354263.74, R-sq = 0.52, MAPE = 0.99 


In [145]:
from catboost import CatBoostRegressor


In [146]:
from tqdm.notebook import tqdm

### CatBoost

In [147]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn import metrics 

# from catboost import CatBoostRegressor

# VAL_SIZE   = 0.33
# N_FOLDS    = 10

# # CATBOOST
# ITERATIONS = 4000
# LR         = 0.05

# RANDOM_SEED = 42
# VERSION=3

# X = train.drop(['itemPrice'], axis = 1)
# y = train['itemPrice']

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = VAL_SIZE, random_state=RANDOM_SEED)
# X_train.nunique()
# cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

# model = CatBoostRegressor(iterations = ITERATIONS,
#                           learning_rate = LR,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE']
#                          )
# #model.fit(X_train, y_train,
#  #        cat_features=cat_features_ids,
#   #       eval_set=(X_test, y_test),
#    #      verbose_eval=100,
#     #     use_best_model=True,
#      #    plot=True
#       #   )

# print_regression_metrics(y_test, y_pred)

In [148]:
corr_matrix = train.corr()
corr_features = np.abs(corr_matrix[np.abs(corr_matrix.itemPrice)>0.00].itemPrice).sort_values(ascending=False)
print("Отсортированный в порядке убывания список коррелирующих с целевой переменной признаков:\n",corr_features[1:])
for index in corr_matrix.index:
    for column in corr_matrix.columns:
        if corr_matrix[index][column]>0.7 and index!=column:
            print (index, column, corr_matrix[index][column])

Отсортированный в порядке убывания список коррелирующих с целевой переменной признаков:
 enginePower           0.489169
engineDisplacement    0.362523
productionDate        0.273587
mileage               0.267104
Mercedes-Benz         0.216472
                        ...   
GMC                   0.005191
Haval                 0.004616
Isuzu                 0.004466
MINI                  0.004040
Iveco                 0.000600
Name: itemPrice, Length: 85, dtype: float64
engineDisplacement enginePower 0.8445516605296565
enginePower engineDisplacement 0.8445516605296565


In [149]:
train.corr()

,bodyType,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,itemPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new,Alfa Romeo,Audi,BAW,BMW,...,SEAT,Saab,Skoda,Smart,SsangYong,Subaru,Suzuki,Toyota,Volkswagen,Volvo,Vortex,other,ВАЗ (LADA),ВИС,ГАЗ,ЗАЗ,ИЖ,Москвич,ТагАЗ,УАЗ
bodyType,1.000000,-0.196810,-0.024324,0.045555,-0.056948,0.053731,0.050117,-0.036471,0.031902,-0.130125,0.190775,-0.187924,0.020678,0.133199,0.231825,-0.010978,-0.001529,-0.031531,0.033767,-0.056718,...,0.005825,-0.008878,0.066989,0.032929,-0.004083,0.008115,-0.012443,-0.032986,-0.017296,-0.033950,-0.010375,0.045631,-0.025874,0.041677,0.259132,-0.026397,0.029251,-0.007955,-0.005720,0.041519
numberOfDoors,-0.196810,1.000000,0.125692,0.014549,0.053635,-0.031884,0.013146,-0.069886,0.041008,0.084512,-0.109979,0.452752,-0.000581,0.116274,0.072478,0.002711,-0.011811,-0.023951,-0.050005,-0.033642,...,0.010885,-0.014215,0.145812,-0.064647,0.061957,0.044951,0.030879,0.031896,-0.014916,0.012788,0.007960,-0.034741,-0.020721,-0.065205,-0.291966,-0.070630,-0.018942,0.005645,-0.019737,0.039853
productionDate,-0.024324,0.125692,1.000000,-0.057443,0.132913,-0.429865,-0.130748,-0.403813,0.273587,-0.063837,-0.101738,0.208810,0.034842,0.223659,0.108925,0.003202,-0.023918,-0.007206,0.003249,0.005033,...,-0.004614,-0.036641,0.090679,0.011434,0.019405,-0.047183,-0.020735,-0.020379,0.012106,-0.037682,0.006430,-0.040638,-0.108252,0.003911,-0.095996,-0.081443,-0.039914,-0.139870,0.001303,-0.001571
engineDisplacement,0.045555,0.014549,-0.057443,1.000000,0.844552,0.061170,-0.033312,0.056125,0.362523,0.155138,-0.384179,0.279940,0.018186,0.108578,0.130601,0.012172,-0.000978,0.038177,0.003096,0.134723,...,-0.015892,-0.002806,-0.123345,-0.054547,-0.005016,0.000444,-0.044468,0.103309,-0.095294,0.016595,-0.011163,0.024791,-0.190385,-0.015380,0.066242,-0.042370,-0.021611,-0.029249,0.003324,0.030008
enginePower,-0.056948,0.053635,0.132913,0.844552,1.000000,-0.060428,-0.028484,0.013217,0.489169,0.180606,-0.430052,0.347842,0.021373,0.081849,0.045137,0.021276,0.002086,0.109476,-0.013597,0.246627,...,-0.010902,0.012567,-0.088652,-0.041645,-0.012282,0.019258,-0.044315,0.036341,-0.080788,0.043892,-0.009937,0.013325,-0.243964,-0.021219,-0.098293,-0.050299,-0.032447,-0.043212,-0.003060,-0.041431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ЗАЗ,-0.026397,-0.070630,-0.081443,-0.042370,-0.050299,-0.034682,-0.007373,-0.019637,-0.020649,-0.010558,0.038191,-0.039052,0.002575,-0.038100,-0.017399,-0.000161,-0.000886,-0.007644,-0.000660,-0.010833,...,-0.000957,-0.001478,-0.007178,-0.001564,-0.003418,-0.003799,-0.003601,-0.009665,-0.010515,-0.005295,-0.000957,-0.002467,-0.010341,-0.000861,-0.006935,1.000000,-0.001254,-0.001711,-0.000911,-0.002976
ИЖ,0.029251,-0.018942,-0.039914,-0.021611,-0.032447,-0.013190,-0.005709,-0.002268,-0.017376,-0.006546,0.030546,-0.051037,-0.000140,-0.038053,-0.013473,-0.002221,-0.000686,-0.005919,-0.000511,-0.008389,...,-0.000741,-0.001144,-0.005558,-0.001211,-0.002647,-0.002942,-0.002789,-0.007484,-0.008142,-0.004100,-0.000741,-0.001910,-0.008007,-0.000667,-0.005370,-0.001254,1.000000,-0.001325,-0.000705,-0.002305
Москвич,-0.007955,0.005645,-0.139870,-0.029249,-0.043212,-0.026706,0.002947,-0.011458,-0.022662,-0.025960,0.041689,-0.040795,-0.000735,-0.036404,-0.018388,0.003588,-0.000937,-0.008078,-0.000698,-0.011449,...,-0.001012,-0.001562,-0.007586,-0.001653,-0.003613,-0.004015,-0.003806,-0.010214,-0.011112,-0.005596,-0.001012,-0.002607,-0.010929,-0.000910,-0.007329,-0.001711,-0.001325,1.000000,-0.000962,-0.003145
ТагАЗ,-0.005720,-0.019737,0.001303,0.003324,-0.003060,-0.004119,-0.004147,0.003203,-0.008643,0.001614,0.011028,0.011130,0.003773,0.018086,0.000095,-0.003616,-0.000498,-0.004299,-0.000371,-0.006093,...,-0.000538,-0.000831,-0.004037,-0.000880,-0.001923,-0.002137,-0.002025,-0

### собрал все предыдущие шаги в одной ячейке чтобы посмотреть, что будет, если дропнуть пару признаокв с учетом корреляции

In [150]:
url_train="https://drive.google.com/u/0/uc?id=1RhXDaw02wlVtvUggA9C81i78vDTM8u14&export=download"
train = pd.read_csv(url_train) # мой подготовленный датасет для обучения модели
#test = pd.read_csv('test.csv')
#sample_submission = pd.read_csv('sample_submission.csv')

#На первый взгляд видим следующее:
#1. enginePower- нужно будет убрать "л.с."
#2. description- предлагаю полностью убрать это столбец
#3. autoCatalogUrl, status, closingReason, canonicalUrl, avitoId, Аудиосистема, Фары - можно спокойно удалять. Такие значения
# как аудиосистема или фары, практически не вляют на ценник, да и информации по ним мало.
#4. С остальными полями будем разбираться дальше.
train.drop(['autoCatalogUrl', 'status', 'closingReason','canonicalUrl','avitoId','Аудиосистема','Фары', 'description'], inplace = True, axis = 1)

train.dropna(subset = ['bodyType'],inplace = True)

#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10}

#Заменяем
train['bodyType'] = train['bodyType'].map(dict_bodyType)

#В столбце "Руль" есть пропуски, убираем.
print(train.Руль.value_counts())
print(train.Руль.isna().sum())
fs = train[train.Руль.isnull()]

train.loc[(train['brand'] == "BMW") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Renault") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Volkswagen") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Peugeot") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Chevrolet") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Audi") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Opel") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Lexus") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Skoda") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Mercedes-Benz") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train.Руль.isnull()) , 'Руль'] = "Правый"

#Нашел статью на хабре, пробуем https://habr.com/ru/post/456294/  , а то в прошлый раз все словарями делал. 
LE = LabelEncoder()
LE.fit(train['color'])
train['Color_new'] = LE.transform(train['color'])
train.drop(['color'],axis = 1, inplace = True)
LE.fit(train['vehicleTransmission'])
train['vehicleTransmission_new'] = LE.transform(train['vehicleTransmission'])
train.drop(['vehicleTransmission'],axis = 1, inplace = True)
LE.fit(train['Привод'])
train['Привод_new'] = LE.transform(train['Привод'])
train.drop(['Привод'],axis = 1, inplace = True)
LE = LabelEncoder()
#LE.fit(train['Руль']) Выдает ошибку, может кто то поймет как с ней бороться
#train['Руль_new'] = LE.transform(train['Руль'])
#train.drop(['Руль'],axis = 1, inplace = True)
LE.fit(train['Состояние'])
train['Состояние_new'] = LE.transform(train['Состояние'])
train.drop(['Состояние'],axis = 1, inplace = True)
LE.fit(train['name'])
train['name_new'] = LE.transform(train['name'])
train.drop(['name'],axis = 1, inplace = True)
LE.fit(train['fuelType'])
train['fuelType_new'] = LE.transform(train['fuelType'])
train.drop(['fuelType'],axis = 1, inplace = True)
LE.fit(train['typeOfTrade'])
train['typeOfTrade_new'] = LE.transform(train['typeOfTrade'])
train.drop(['typeOfTrade'],axis = 1, inplace = True)


#Чистим от л.с.
train['enginePower'] = train['enginePower'].apply(lambda x: float(str(x).split()[0]))

train.dropna(subset = ['engineDisplacement'],inplace = True)

train.loc[(train["engineDisplacement"] == "6.0+") , 'engineDisplacement'] = "7.0"

train.enginePower = train.enginePower.astype(np.int32)


train[train.numberOfDoors.isna()]
train.dropna(subset = ['numberOfDoors'],inplace = True)

train.numberOfDoors = train.numberOfDoors.astype(np.int32)

train.loc[(train["Владельцы"] == "4+") , 'Владельцы'] = 4

train.Владельцы = train.Владельцы.astype(np.int32)


#Дропаем оставшиеся признаки

train.drop(['Электростеклоподъемники', 'Усилитель руля', 'Климат-контроль','Салон','Диски','marketPrice'], inplace = True, axis = 1)


### разбираемся с brand
all_brands = train.brand.value_counts().index
top_brands = list(all_brands)[:top_brands_count]
brands_to_throw_away = list(set(all_brands) - set(top_brands))
train.loc[train['brand'].isin(brands_to_throw_away), 
             'brand'] = 'other'


brands_df = pd.get_dummies(train.brand, drop_first=True)  
train = pd.concat([train, brands_df],axis = 1)

#дропаем первоначальную колонку brand
train.drop(['brand'], inplace = True, axis = 1)

#переводим engineDisplacement из obj в float
train.engineDisplacement = train.engineDisplacement.astype(np.float)

#забыл про руль

dict_rul={"Левый": 1, "Правый": 2}
train['Руль'] = train['Руль'].map(dict_rul)

train.info()

Левый     36017
Правый     1145
Name: Руль, dtype: int64
194
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 86 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   numberOfDoors            37121 non-null  int32  
 2   productionDate           37121 non-null  int64  
 3   engineDisplacement       37121 non-null  float64
 4   enginePower              37121 non-null  int32  
 5   mileage                  37121 non-null  int64  
 6   Руль                     37121 non-null  int64  
 7   Владельцы                37121 non-null  int32  
 8   itemPrice                37121 non-null  int64  
 9   Color_new                37121 non-null  int64  
 10  vehicleTransmission_new  37121 non-null  int64  
 11  Привод_new               37121 non-null  int64  
 12  Состояние_new            37121 non-null  int64  
 13  name_new       

### так как engineDisplacement enginePower сильно скорелированы между собой, можно попытаться удалить один из признаков. Оставляю enginePower, так он сильнее влияет на целевую переменную

In [151]:
train.drop(['е'], inplace = True, axis = 1)


KeyError: "['е'] not found in axis"

### Строим модели еще раз

In [152]:
X = train.drop(['itemPrice'], axis = 1)
y = train['itemPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

### линейная регрессия без нормализации

# training the model
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

print_regression_metrics(y_test, y_pred)

RMSE = 647937.15, MAE = 355612.45, R-sq = 0.52, MAPE = 0.99 


In [153]:
### Случайный лес без нормализации
RANDOM_SEED = 42
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict

forest = RandomForestRegressor(n_estimators=10, random_state=RANDOM_SEED) #max_depth = 15, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100, verbose = 100)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')
print_regression_metrics(y_test, y_pred)


RMSE = 535126.75, MAE = 152630.49, R-sq = 0.67, MAPE = 0.33 


In [154]:
X = train.drop(['itemPrice'], axis = 1)
y = train['itemPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

# нормализуем X_train
transformer = RobustScaler()
X_train = transformer.fit_transform(X_train)


# normalising X_test
transformer = RobustScaler()
X_test = transformer.fit_transform(X_test)

# training the model
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

print_regression_metrics(y_test, y_pred)

RMSE = 647477.11, MAE = 354263.74, R-sq = 0.52, MAPE = 0.99 


In [155]:
### Случайный лес с нормализацией
RANDOM_SEED = 42
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict

forest = RandomForestRegressor(n_estimators=10, random_state=RANDOM_SEED) #max_depth = 15, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100, verbose = 100)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')
print_regression_metrics(y_test, y_pred)

RMSE = 508087.97, MAE = 155086.02, R-sq = 0.70, MAPE = 0.35 


### Это все было на выборке всех машин, посмотрим как это будет выглядеть только на BMW

In [156]:
url_train="https://drive.google.com/u/0/uc?id=1RhXDaw02wlVtvUggA9C81i78vDTM8u14&export=download"
train = pd.read_csv(url_train) # мой подготовленный датасет для обучения модели
#test = pd.read_csv('test.csv')
#sample_submission = pd.read_csv('sample_submission.csv')

#На первый взгляд видим следующее:
#1. enginePower- нужно будет убрать "л.с."
#2. description- предлагаю полностью убрать это столбец
#3. autoCatalogUrl, status, closingReason, canonicalUrl, avitoId, Аудиосистема, Фары - можно спокойно удалять. Такие значения
# как аудиосистема или фары, практически не вляют на ценник, да и информации по ним мало.
#4. С остальными полями будем разбираться дальше.
train.drop(['autoCatalogUrl', 'status', 'closingReason','canonicalUrl','avitoId','Аудиосистема','Фары', 'description'], inplace = True, axis = 1)

train.dropna(subset = ['bodyType'],inplace = True)

#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10}

#Заменяем
train['bodyType'] = train['bodyType'].map(dict_bodyType)

#В столбце "Руль" есть пропуски, убираем.
print(train.Руль.value_counts())
print(train.Руль.isna().sum())
fs = train[train.Руль.isnull()]

train.loc[(train['brand'] == "BMW") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Renault") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Volkswagen") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Peugeot") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Chevrolet") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Audi") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Opel") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Lexus") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Skoda") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Mercedes-Benz") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train.Руль.isnull()) , 'Руль'] = "Правый"

#Нашел статью на хабре, пробуем https://habr.com/ru/post/456294/  , а то в прошлый раз все словарями делал. 
LE = LabelEncoder()
LE.fit(train['color'])
train['Color_new'] = LE.transform(train['color'])
train.drop(['color'],axis = 1, inplace = True)
LE.fit(train['vehicleTransmission'])
train['vehicleTransmission_new'] = LE.transform(train['vehicleTransmission'])
train.drop(['vehicleTransmission'],axis = 1, inplace = True)
LE.fit(train['Привод'])
train['Привод_new'] = LE.transform(train['Привод'])
train.drop(['Привод'],axis = 1, inplace = True)
LE = LabelEncoder()
#LE.fit(train['Руль']) Выдает ошибку, может кто то поймет как с ней бороться
#train['Руль_new'] = LE.transform(train['Руль'])
#train.drop(['Руль'],axis = 1, inplace = True)
LE.fit(train['Состояние'])
train['Состояние_new'] = LE.transform(train['Состояние'])
train.drop(['Состояние'],axis = 1, inplace = True)
LE.fit(train['name'])
train['name_new'] = LE.transform(train['name'])
train.drop(['name'],axis = 1, inplace = True)
LE.fit(train['fuelType'])
train['fuelType_new'] = LE.transform(train['fuelType'])
train.drop(['fuelType'],axis = 1, inplace = True)
LE.fit(train['typeOfTrade'])
train['typeOfTrade_new'] = LE.transform(train['typeOfTrade'])
train.drop(['typeOfTrade'],axis = 1, inplace = True)


#Чистим от л.с.
train['enginePower'] = train['enginePower'].apply(lambda x: float(str(x).split()[0]))

train.dropna(subset = ['engineDisplacement'],inplace = True)

train.loc[(train["engineDisplacement"] == "6.0+") , 'engineDisplacement'] = "7.0"

train.enginePower = train.enginePower.astype(np.int32)


train[train.numberOfDoors.isna()]
train.dropna(subset = ['numberOfDoors'],inplace = True)

train.numberOfDoors = train.numberOfDoors.astype(np.int32)

train.loc[(train["Владельцы"] == "4+") , 'Владельцы'] = 4

train.Владельцы = train.Владельцы.astype(np.int32)


#Дропаем оставшиеся признаки

train.drop(['Электростеклоподъемники', 'Усилитель руля', 'Климат-контроль','Салон','Диски','marketPrice'], inplace = True, axis = 1)


### разбираемся с brand
# all_brands = train.brand.value_counts().index
# top_brands = list(all_brands)[:top_brands_count]
# brands_to_throw_away = list(set(all_brands) - set(top_brands))
# train.loc[train['brand'].isin(brands_to_throw_away), 
#              'brand'] = 'other'


# brands_df = pd.get_dummies(train.brand, drop_first=True)  
# train = pd.concat([train, brands_df],axis = 1)

# #дропаем первоначальную колонку brand
# train.drop(['brand'], inplace = True, axis = 1)

#переводим engineDisplacement из obj в float
train.engineDisplacement = train.engineDisplacement.astype(np.float)

#забыл про руль

dict_rul={"Левый": 1, "Правый": 2}
train['Руль'] = train['Руль'].map(dict_rul)

#Оставим одни значения BMW
train = train[train['brand'].isin(['BMW'])]
train.drop(['brand'], inplace = True, axis = 1)

train.info()

Левый     36017
Правый     1145
Name: Руль, dtype: int64
194
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2509 entries, 16 to 37400
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 2509 non-null   int64  
 1   numberOfDoors            2509 non-null   int32  
 2   productionDate           2509 non-null   int64  
 3   engineDisplacement       2509 non-null   float64
 4   enginePower              2509 non-null   int32  
 5   mileage                  2509 non-null   int64  
 6   Руль                     2509 non-null   int64  
 7   Владельцы                2509 non-null   int32  
 8   itemPrice                2509 non-null   int64  
 9   Color_new                2509 non-null   int64  
 10  vehicleTransmission_new  2509 non-null   int64  
 11  Привод_new               2509 non-null   int64  
 12  Состояние_new            2509 non-null   int64  
 13  name_new       

### Строим модели еще раз

In [157]:
X = train.drop(['itemPrice'], axis = 1)
y = train['itemPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

### линейная регрессия без нормализации

# training the model
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

print_regression_metrics(y_test, y_pred)

RMSE = 627258.46, MAE = 412938.32, R-sq = 0.64, MAPE = 0.84 


In [158]:
### Случайный лес без нормализации
RANDOM_SEED = 42
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict

forest = RandomForestRegressor(n_estimators=10, random_state=RANDOM_SEED) #max_depth = 15, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100, verbose = 100)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')
print_regression_metrics(y_test, y_pred)

RMSE = 374288.65, MAE = 194008.69, R-sq = 0.87, MAPE = 0.38 
